### 1. Constrói base de dados `custom_fields`

- Para cada lead, constrói a base de dados
- Cada chave é um lead que possui informações de campos personalizados no Exact


In [37]:
import requests
import json


def request_API(skip_num):
    url = "https://api.exactspotter.com/v3/CustomFieldsLeads"
    headers = {
        "Content-Type": "application/json",
        "token_exact": "35fc1bd4-7d6e-4f40-bffd-123d2f46054a",
    }
    params = {"$skip": skip_num, "$orderby": "leadId"}

    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    return data


all_leads = []
skip = 0
while True:
    data = request_API(skip)
    leads_fields = data["value"]
    if len(leads_fields) == 0:
        break
    all_leads.extend(leads_fields)
    skip += 500

# Filter and process the important data
filtered_leads = []
for lead in all_leads:
    filtered_lead = {
        "key": lead.get("key"),
        "id_lead": lead.get("leadId"),
        "value": lead.get("value"),
    }

    # Check if 'options' key exists and is not empty
    if "options" in lead and lead["options"]:
        # Assuming you want to include the first option only
        option = lead["options"][0]
        filtered_lead["value"] = option.get("value")

    filtered_leads.append(filtered_lead)

# Create a dictionary to group the information by "id_lead"
grouped_leads = {}
for lead in filtered_leads:
    id_lead = lead["id_lead"]
    if id_lead not in grouped_leads:
        grouped_leads[id_lead] = {}

    grouped_leads[id_lead][lead["key"]] = lead["value"]

# Create a list of dictionaries in the desired format
reorganized_leads = [
    {"lead_id": id_lead, **data} for id_lead, data in grouped_leads.items()
]

# Save the result to a JSON file
with open("custom_fields.json", "w") as file:
    json.dump(reorganized_leads, file)

### 2. Constrói base de dados de `leads`

- Para cada lead, constrói a base de dados
- Organiza as informações da forma relevante para o e-mail


In [1]:
import requests
import json


def request_API(skip_num):
    url = "https://api.exactspotter.com/v3/Leads"
    headers = {
        "Content-Type": "application/json",
        "token_exact": "35fc1bd4-7d6e-4f40-bffd-123d2f46054a",
    }
    params = {"$skip": skip_num, "$orderby": "id"}

    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    return data


all_leads = []
skip = 0
while True:
    data = request_API(skip)
    leads_fields = data["value"]
    if len(leads_fields) == 0:
        break
    all_leads.extend(leads_fields)
    skip += 500

# Filter and process the important data
filtered_leads = []
for lead in all_leads:
    # Construindo objeto de logradouro
    if lead.get("street") and lead.get("number"):
        street = lead.get("street")
        number = lead.get("number")

        logradouro = f"{street} - {number}"

    # Construindo objeto de cidade, estado, país
    if lead.get("city") and lead.get("state") and lead.get("country"):
        city = lead.get("city")
        state = lead.get("state")
        country = lead.get("country")

        cidade_estado_pais = f"{city} - {state} - {country}"

    # Construindo objeto de lead filtrado
    filtered_lead = {
        "id": lead.get("id"),
        "phone_1": lead.get("phone1"),
        "phone_2": lead.get("phone2"),
        "cnpj": lead.get("cnpj"),
        "logradouro": logradouro,
        "cep": lead.get("cep"),
        "district": lead.get("district"),
        "cidade_estado_pais": cidade_estado_pais,
        "public_link": lead.get("publicLink"),
    }

    # Check if 'industry' key exists and is not empty
    if "industry" in lead and lead["industry"]:
        # Assuming you want to include the first option only
        filtered_lead["industry"] = lead["industry"]["value"]

    # Check if 'sdr' key exists and is not empty
    if "sdr" in lead and lead["sdr"]:
        # Assuming you want to include the first option only
        sdr_name = lead["sdr"]["name"]
        sdr_last_name = lead["sdr"]["lastName"]
        filtered_lead["sdr"] = f"{sdr_name} {sdr_last_name}"

    filtered_leads.append(filtered_lead)

# Save filtered leads to a JSON file
with open("leads_info.json", "w") as file:
    json.dump(filtered_leads, file)

1794808


KeyError: 0

## 3. Faz o merge das bases de `Campos Personalizados` e `Informações Gerais` dos leads

- O resultado é um arquivo `leads_complete`
- Com as informações organizadas para atualizar a planilha


In [16]:
import json

# Load the data from the JSON files
with open("custom_fields.json", "r") as custom_fields_file:
    custom_fields_data = json.load(custom_fields_file)

with open("leads_info.json", "r") as exact_leads_file:
    exact_leads_data = json.load(exact_leads_file)

# Create a dictionary to store the data with "lead_id" as the key
lead_id_dict = {entry["lead_id"]: entry for entry in custom_fields_data}

# Join the data using the "id" from exact_leads_data as the key
result_data = [
    dict(exact_lead, **lead_id_dict.get(exact_lead["id"], {}))
    for exact_lead in exact_leads_data
]

# Remove the "lead_id" from the merged data
for entry in result_data:
    entry.pop("lead_id", None)

# Write the result into a new JSON file
with open("leads_completo.json", "w") as output_file:
    json.dump(result_data, output_file, indent=4)

## 4. Atualiza a planilha com informações


In [63]:
import gspread
import json


# Função para converter índice da coluna em letra
def index_to_letter(index):
    quotient = (index - 1) // 26
    remainder = (index - 1) % 26
    letter = chr(65 + remainder)
    if quotient > 0:
        return index_to_letter(quotient) + letter
    return letter


# Autenticando e acessando a planilha do Google Sheets
gc = gspread.service_account(filename="./token.json")
spreadsheet = gc.open("Agendamento Especialistas 2023")
worksheet = spreadsheet.sheet1

# Lendo o conteúdo do arquivo exact_custom_fields.json
with open("leads_completo.json", "r") as file:
    data_json = file.read()

# Convertendo o conteúdo do arquivo .json para uma lista de dicionários
data_list = json.loads(data_json)

# Mapeamento de colunas da planilha pelo nome
column_mapping = {
    # "id": "Id Exact",
    "name": "Empresa",
    "phone_1": "Telefone 1",
    "phone_2": "Telefone 2",
    "cnpj": "CNPJ",
    "logradouro": "Logradouro",
    "cep": "CEP",
    "district": "Bairro",
    "cidade_estado_pais": "Cidado Estado País",
    "public_link": "Link Público",
    "industry": "Indústria",
    "sdr": "Nome - SDR",
    "_operadoraatual": "Operadora Atual",
    "_valoratual": "Valor Atual",
    "_operadorasugerida": "Operadora Sugerida",
    "_valorproposta": "Valor Proposta",
    "_temperaturadeoportunidade": "Temperatura Oportunidade",
    "_cotacao": "Cotação",
    "_clienteabsoluta": "Cliente Absoluta",
    "_mesdevigenciaaniversariodaapolice1": "Mês Vigência",
    "_operadorasugerida": "Plano Sugerido",
}

# Obtendo os dados da planilha
data_sheet = worksheet.get_all_records()

batch_update_requests = []

# Localizar todas as células correspondentes aos nomes das colunas
cell_mapping = {}
for json_key, sheet_column in column_mapping.items():
    cell = worksheet.find(sheet_column)
    cell_mapping[json_key] = cell

# Coletar todas as atualizações em uma lista
batch_update_data = []
for idx, row in enumerate(data_sheet, start=1):
    id_exact = row["Id Exact"]
    if id_exact:
        for data in data_list:
            if data.get("id") == int(id_exact):
                for json_key, cell in cell_mapping.items():
                    if json_key in data:
                        col_letter = index_to_letter(cell.col)
                        cell_range = f"{col_letter}{idx + 1}"
                        batch_update_data.append(
                            {"range": cell_range, "values": [[data[json_key]]]}
                        )
                print(f"Id Exact: {id_exact} | ID: {data['id']} | Dados atualizados.")
                break
        else:
            print(f"Id Exact: {id_exact} | Dados não encontrados no arquivo .json.")
    else:
        print(f"Linha {idx} - Id Exact está vazio. Ignorando a atualização.")

# Batch update the cells
if batch_update_data:
    worksheet.batch_update(batch_update_data)

print("Atualização concluída!")

Id Exact: 2021483 | ID: 2021483 | Dados atualizados.
Id Exact: 5083135 | ID: 5083135 | Dados atualizados.
Linha 3 - Id Exact está vazio. Ignorando a atualização.
Linha 4 - Id Exact está vazio. Ignorando a atualização.
Linha 5 - Id Exact está vazio. Ignorando a atualização.
Id Exact: 2034209 | ID: 2034209 | Dados atualizados.
Linha 7 - Id Exact está vazio. Ignorando a atualização.
Linha 8 - Id Exact está vazio. Ignorando a atualização.
Id Exact: 2028888 | ID: 2028888 | Dados atualizados.
Id Exact: 12430834 | ID: 12430834 | Dados atualizados.
Id Exact: 6704198 | ID: 6704198 | Dados atualizados.
Id Exact: 2854220 | ID: 2854220 | Dados atualizados.
Id Exact: 2024509 | ID: 2024509 | Dados atualizados.
Id Exact: 2854173 | ID: 2854173 | Dados atualizados.
Id Exact: 2028323 | ID: 2028323 | Dados atualizados.
Id Exact: 4888459 | ID: 4888459 | Dados atualizados.
Id Exact: 2024267 | ID: 2024267 | Dados atualizados.
Linha 18 - Id Exact está vazio. Ignorando a atualização.
Id Exact: 2029968 | ID: 2

### 5. Base de comparação de Nome - ID

- Atualiza a planilha na aba `Dados Leads`
- Vai servir pra não precisar colocar o `Id Exact`
- Assim, deve ser preenchido o nome da empresa exatamente como está no Exact


In [40]:
import requests
import gspread
import json


def request_API(skip_num):
    url = "https://api.exactspotter.com/v3/Leads"
    headers = {
        "Content-Type": "application/json",
        "token_exact": "35fc1bd4-7d6e-4f40-bffd-123d2f46054a",
    }
    params = {"$skip": skip_num, "$orderby": "id"}

    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    return data


all_leads = []
skip = 0
while True:
    data = request_API(skip)
    leads_fields = data["value"]
    if len(leads_fields) == 0:
        break
    all_leads.extend(leads_fields)
    skip += 500

# Filter and process the important data
filtered_leads = []
for lead in all_leads:
    # Construindo objeto de lead filtrado
    filtered_lead = {
        "id": lead.get("id"),
        "name": lead.get("lead"),
    }

    filtered_leads.append(filtered_lead)

# Autenticando e acessando a planilha do Google Sheets
gc = gspread.service_account(filename="./token.json")
spreadsheet = gc.open(
    "Agendamento Especialistas 2023"
)  # Substitua "NOME_DA_SUA_PLANILHA" pelo nome correto da sua planilha
worksheet = spreadsheet.worksheet(
    "Dados Leads"
)  # Substitua "Dados Leads" pelo nome correto da aba onde deseja atualizar os dados

# Limpando a planilha antes de atualizar os dados
worksheet.clear()

# Atualizando os dados na aba "Dados Leads" com base no arquivo JSON
header_row = ["name", "id"]  # Cabeçalho das colunas
data_to_update = [header_row]  # Lista que conterá todos os dados para atualizar

for lead_data in filtered_leads:
    lead_id = lead_data["id"]
    lead_name = lead_data["name"]
    data_to_update.append([lead_name, lead_id])

# Atualizando toda a planilha com os dados do arquivo JSON
worksheet.update(data_to_update)

print("Atualização concluída!")

KeyboardInterrupt: 